In [6]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs

import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get("https://www.amazon.com/")

[WDM] - Downloading: 100%|██████████| 6.68M/6.68M [00:00<00:00, 11.4MB/s]


In [23]:
a = driver.find_element_by_css_selector('#twotabsearchtextbox')

a.click()
a.send_keys('nature republic')
a.send_keys(Keys.ENTER)
time.sleep(1)
soup = bs(driver.page_source, 'html.parser')
search = soup.select('span > a.a-link-normal.s-no-outline')
time.sleep(1)

n=0
for i in search:
    if (n < 8):
        n= n+1
        continue
    elif(n > 10):
        break
    driver.get('https://www.amazon.com' + i['href'])
    time.sleep(0.3)
    driver.back()
    time.sleep(0.3)
    n= n+1